In [4]:
import sys
sys.path.append('/Users/syesw/Desktop/reinforcement_learning_tutorial')
sys.path.append('/Users/syesw/Desktop/reinforcement_learning_tutorial/cfralgo')

In [5]:
from cfralgo.common.constants import CARDS_DEALINGS, DUDO_RESULT_MAP
from cfralgo.games.kuhn import KuhnRootChanceGameState
from cfralgo.games.algorithms import ChanceSamplingCFR, VanillaCFR

In [6]:
root = KuhnRootChanceGameState(CARDS_DEALINGS)
print("当前节点所在的信息集I: {0}".format(root.inf_set()))
print("chance节点的合法行动: {0}".format(root.actions))
print("chance节点的子节点: {0}".format(root.children))
print("\n")

player1 = root.sample_one()
print("chance节点发牌: {0}".format(player1.cards))
print("当前玩家1节点所在的信息集I: {0}".format(player1.inf_set()))
print("当前玩家1节点行动路径h: {0}".format(player1.actions_history))
print("当前玩家1节点的合法行动: {0}".format(player1.actions))
print("当前玩家1节点的子节点: {0}".format(player1.children))
print("\n")

player2 = player1.children['BET']
print("玩家1选择bet")
print("当前玩家2节点所在的信息集I: {0}".format(player2.inf_set()))
print("当前玩家2节点行动路径h: {0}".format(player2.actions_history))
print("当前玩家2节点的合法行动: {0}".format(player2.actions))
print("当前玩家2节点的子节点: {0}".format(player2.children))
print("\n")

player1 = player2.children['CALL']
print("玩家2选择call")
print("当前玩家1节点所在的信息集I: {0}".format(player1.inf_set()))
print("当前玩家1节点行动路径h: {0}".format(player1.actions_history))
print("当前玩家1节点的合法行动: {0}".format(player1.actions))
print("当前玩家1节点的子节点: {0}".format(player1.children))
print("\n")

print("是否已经到达terminal节点: {0}".format(player1.is_terminal()))
print("terminal节点的价值: {0}".format(player1.evaluation()))

当前节点所在的信息集I: .
chance节点的合法行动: ['KQ', 'KJ', 'QK', 'QJ', 'JK', 'JQ']
chance节点的子节点: {'KQ': <cfralgo.games.kuhn.KuhnPlayerMoveGameState object at 0x1082db5e0>, 'KJ': <cfralgo.games.kuhn.KuhnPlayerMoveGameState object at 0x1082dba60>, 'QK': <cfralgo.games.kuhn.KuhnPlayerMoveGameState object at 0x1082db7c0>, 'QJ': <cfralgo.games.kuhn.KuhnPlayerMoveGameState object at 0x1082db5b0>, 'JK': <cfralgo.games.kuhn.KuhnPlayerMoveGameState object at 0x1082dbc40>, 'JQ': <cfralgo.games.kuhn.KuhnPlayerMoveGameState object at 0x1082dba30>}


chance节点发牌: JQ
当前玩家1节点所在的信息集I: .J.
当前玩家1节点行动路径h: []
当前玩家1节点的合法行动: ['BET', 'CHECK']
当前玩家1节点的子节点: {'BET': <cfralgo.games.kuhn.KuhnPlayerMoveGameState object at 0x1082cf760>, 'CHECK': <cfralgo.games.kuhn.KuhnPlayerMoveGameState object at 0x1082cf490>}


玩家1选择bet
当前玩家2节点所在的信息集I: .Q.BET
当前玩家2节点行动路径h: ['BET']
当前玩家2节点的合法行动: ['FOLD', 'CALL']
当前玩家2节点的子节点: {'FOLD': <cfralgo.games.kuhn.KuhnPlayerMoveGameState object at 0x1082cf880>, 'CALL': <cfralgo.games.kuhn.KuhnPlayerMoveGame

In [7]:
chance_sampling_cfr = ChanceSamplingCFR(root)

# 玩家1信息集(12)+玩家2信息集(12)+chance信息集(1)
chance_sampling_cfr.sigma

information_set = list(chance_sampling_cfr.sigma.keys())
print("一共有{0}个信息集".format(len(information_set)))
print("这些信息集为：")
print("1个chance节点, 6个玩家1节点, 6个玩家2节点, 6个玩家1的终止状态节点, 9个玩家2的终止状态节点")
information_set

一共有28个信息集
这些信息集为：
1个chance节点, 6个玩家1节点, 6个玩家2节点, 6个玩家1的终止状态节点, 9个玩家2的终止状态节点


['.',
 '.K.',
 '.Q.BET',
 '.K.BET.FOLD',
 '.K.BET.CALL',
 '.Q.CHECK',
 '.K.CHECK.BET',
 '.Q.CHECK.BET.CALL',
 '.Q.CHECK.BET.FOLD',
 '.K.CHECK.CHECK',
 '.J.BET',
 '.J.CHECK',
 '.J.CHECK.BET.CALL',
 '.J.CHECK.BET.FOLD',
 '.Q.',
 '.K.BET',
 '.Q.BET.FOLD',
 '.Q.BET.CALL',
 '.K.CHECK',
 '.Q.CHECK.BET',
 '.K.CHECK.BET.CALL',
 '.K.CHECK.BET.FOLD',
 '.Q.CHECK.CHECK',
 '.J.',
 '.J.BET.FOLD',
 '.J.BET.CALL',
 '.J.CHECK.BET',
 '.J.CHECK.CHECK']

In [8]:
I = '.Q.BET'
print(chance_sampling_cfr.sigma[I])
print(chance_sampling_cfr.cumulative_regrets[I])
print(chance_sampling_cfr.cumulative_sigma[I])
print(chance_sampling_cfr.nash_equilibrium[I])

{'FOLD': 0.5, 'CALL': 0.5}
{'FOLD': 0.0, 'CALL': 0.0}
{'FOLD': 0.0, 'CALL': 0.0}
{'FOLD': 0.0, 'CALL': 0.0}


In [9]:
'''
# 循环T=100次
# 每次迭代运行_cfr_utility_recursive
def _cfr_utility_recursive(self, state, reach_a, reach_b):
    children_states_utilities = {}

    if state.is_terminal():
        # 如果是终止节点，则把比赛结果返回
        return state.evaluation() # 最基本情况
    
    if state.is_chance():
        # 开启chance_sampling会随机选择一个事件而非全部计算
        return self._cfr_utility_recursive(state.sample_one(), reach_a, reach_b)
    
    value = 0. # 虚拟价值
    for action in state.actions: # 遍历当前节点所有合法动作
        # 更新玩家1和玩家2到达下一节点的概率
        child_reach_a = reach_a * (self.sigma[state.inf_set()][action] if state.to_move == A else 1)
        child_reach_b = reach_b * (self.sigma[state.inf_set()][action] if state.to_move == -A else 1)
        
        # v[i][a] = CFR(ha)
        child_state_utility = self._cfr_utility_recursive(state.play(action), child_reach_a, child_reach_b)
        # v[i] = v[i] + v[i][a]*P(a)
        value +=  self.sigma[state.inf_set()][action] * child_state_utility

        # 记录子状态效用值
        children_states_utilities[action] = child_state_utility
 '''


'\n# 循环T=100次\n# 每次迭代运行_cfr_utility_recursive\ndef _cfr_utility_recursive(self, state, reach_a, reach_b):\n    children_states_utilities = {}\n\n    if state.is_terminal():\n        # 如果是终止节点，则把比赛结果返回\n        return state.evaluation() # 最基本情况\n    \n    if state.is_chance():\n        # 开启chance_sampling会随机选择一个事件而非全部计算\n        return self._cfr_utility_recursive(state.sample_one(), reach_a, reach_b)\n    \n    value = 0. # 虚拟价值\n    for action in state.actions: # 遍历当前节点所有合法动作\n        # 更新玩家1和玩家2到达下一节点的概率\n        child_reach_a = reach_a * (self.sigma[state.inf_set()][action] if state.to_move == A else 1)\n        child_reach_b = reach_b * (self.sigma[state.inf_set()][action] if state.to_move == -A else 1)\n        \n        # v[i][a] = CFR(ha)\n        child_state_utility = self._cfr_utility_recursive(state.play(action), child_reach_a, child_reach_b)\n        # v[i] = v[i] + v[i][a]*P(a)\n        value +=  self.sigma[state.inf_set()][action] * child_state_utility\n\n        # 记录子状态效用值

In [10]:
root = KuhnRootChanceGameState(CARDS_DEALINGS)
chance_sampling_cfr = ChanceSamplingCFR(root)
chance_sampling_cfr.run(iterations = 1000)
chance_sampling_cfr.compute_nash_equilibrium()

nash_strategy = chance_sampling_cfr.nash_equilibrium
#print(nash_strategy)

for key, value in nash_strategy.items():
    print(f"信息集: {key}, 合法行动的策略: {value}")

信息集: ., 合法行动的策略: {'KQ': 0.16666666666666666, 'KJ': 0.16666666666666666, 'QK': 0.16666666666666666, 'QJ': 0.16666666666666666, 'JK': 0.16666666666666666, 'JQ': 0.16666666666666666}
信息集: .K., 合法行动的策略: {'BET': 0.5700454486730775, 'CHECK': 0.4299545513269225}
信息集: .Q.BET, 合法行动的策略: {'FOLD': 0.614011394294567, 'CALL': 0.385988605705433}
信息集: .K.BET.FOLD, 合法行动的策略: {}
信息集: .K.BET.CALL, 合法行动的策略: {}
信息集: .Q.CHECK, 合法行动的策略: {'BET': 0.019864341085271322, 'CHECK': 0.9801356589147288}
信息集: .K.CHECK.BET, 合法行动的策略: {'CALL': 0.9982218446309632, 'FOLD': 0.0017781553690368378}
信息集: .Q.CHECK.BET.CALL, 合法行动的策略: {}
信息集: .Q.CHECK.BET.FOLD, 合法行动的策略: {}
信息集: .K.CHECK.CHECK, 合法行动的策略: {}
信息集: .J.BET, 合法行动的策略: {'FOLD': 0.9984662576687117, 'CALL': 0.0015337423312883436}
信息集: .J.CHECK, 合法行动的策略: {'BET': 0.37215785633115894, 'CHECK': 0.6278421436688412}
信息集: .J.CHECK.BET.CALL, 合法行动的策略: {}
信息集: .J.CHECK.BET.FOLD, 合法行动的策略: {}
信息集: .Q., 合法行动的策略: {'BET': 0.04077147751912672, 'CHECK': 0.9592285224808732}
信息集: .K.BET, 合法行动的

In [11]:
from cfralgo.games.dudo import DudoRootChanceGameState
from cfralgo.common.constants import DUDO_RESULT_MAP

In [12]:
root = DudoRootChanceGameState([1, 1])
chance_sampling_cfr = ChanceSamplingCFR(root)
chance_sampling_cfr.run(iterations = 1000)
chance_sampling_cfr.compute_nash_equilibrium()

nash_strategy = chance_sampling_cfr.nash_equilibrium
#print(nash_strategy)

for key, value in nash_strategy.items():
    print(f"信息集: {key}, 合法行动的策略: {value}")

信息集: ., 合法行动的策略: {(('player1', (1,)), ('player2', (1,))): 0.027777777777777776, (('player1', (1,)), ('player2', (2,))): 0.027777777777777776, (('player1', (1,)), ('player2', (3,))): 0.027777777777777776, (('player1', (1,)), ('player2', (4,))): 0.027777777777777776, (('player1', (1,)), ('player2', (5,))): 0.027777777777777776, (('player1', (1,)), ('player2', (6,))): 0.027777777777777776, (('player1', (2,)), ('player2', (1,))): 0.027777777777777776, (('player1', (2,)), ('player2', (2,))): 0.027777777777777776, (('player1', (2,)), ('player2', (3,))): 0.027777777777777776, (('player1', (2,)), ('player2', (4,))): 0.027777777777777776, (('player1', (2,)), ('player2', (5,))): 0.027777777777777776, (('player1', (2,)), ('player2', (6,))): 0.027777777777777776, (('player1', (3,)), ('player2', (1,))): 0.027777777777777776, (('player1', (3,)), ('player2', (2,))): 0.027777777777777776, (('player1', (3,)), ('player2', (3,))): 0.027777777777777776, (('player1', (3,)), ('player2', (4,))): 0.0277777777